In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.keras.regularizers import l2
import pathlib
import tensorflow 
from tensorflow import keras
from tensorflow.keras.layers import  Conv2D, MaxPooling2D, Flatten, Dense,Dropout,BatchNormalization
import tensorflow.keras 
import pathlib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.utils as utils
from tensorflow.keras.optimizers import Adam as adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adagrad 
from tensorflow.keras.callbacks import  EarlyStopping ,ModelCheckpoint
import tensorflow as tf
from tensorflow.keras import Model
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers import  Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D, Dropout, Input

from tensorflow.keras.applications import VGG16

from tensorflow.keras.applications.resnet50 import preprocess_input
from matplotlib import pyplot
from PIL import Image
from numpy import asarray

import glob
import cv2
from tensorflow.keras import layers

In [3]:
import sys
print("Keras version:", keras.__version__)
print("TensorFlow version:", tf.__version__)
print("Python version:", sys.version)

Keras version: 2.4.0
TensorFlow version: 2.4.0
Python version: 3.7.0 (v3.7.0:1bf9cc5093, Jun 27 2018, 04:59:51) [MSC v.1914 64 bit (AMD64)]


In [4]:
def get_paths():
    classes = []
    for file in sorted(glob.iglob('lfw3/lfw3/lfw-deepfunneled_masked/*/')):
        classes.append(file)
    for i,d in enumerate(classes):
        paths=d+'*.jpg'
        class_=[]     
        for file in sorted(glob.iglob(paths)):
            class_.append(file)
        classes[i]=class_ 
    return classes

In [5]:
def get_all_images(classes):
    allImages=[]
    Labels = []
    count = 0
    for k,d in enumerate(classes[:]):
        for i,sample in enumerate(d):
            org_img = cv2.imread(sample)
            #org_img = org_img.astype('float32')
            org_img = cv2.resize(org_img, (224, 224))
            # org_img=cv2.cvtColor(org_img,cv2.COLOR_BGR2RGB)
            # np.append(allImages, org_img)
            allImages.append(org_img)
            Labels.append(count)
        count = count + 1   
    return np.array(allImages),np.array(Labels)

In [6]:
classes=get_paths()
X,Y=get_all_images(classes)

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.05,shuffle=True, random_state=777)

# Transfer_learning

In [8]:
from keras.models import load_model
# load the model
def Transfer_FacenetModel_withNormlization():
    facenetmodel = tf.keras.models.load_model('facenet_keras.h5',compile=False)
    facenetmodel.load_weights("facenet_keras_weights.h5")
    for layer in facenetmodel.layers[:-50]:
        layer.trainable = False
    inputs = layers.Input(shape=(224,224,3))
    # Augment data.
    augmented = data_augmentation(inputs)
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = facenetmodel(augmented)
    top_model = Dropout(0.5)(top_model)
    top_model = BatchNormalization()(top_model)
    top_model = Flatten(name="flatten")(top_model)
    output_layer = Dense(5750, activation='softmax')(top_model)

    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=inputs, outputs=output_layer)

    return model

Using TensorFlow backend.


In [9]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.Rescaling(scale=1./127.5, offset=-1),
        layers.experimental.preprocessing.Resizing(160, 160),
    ],
    name="data_augmentation",
)

In [10]:
Facenet1=Transfer_FacenetModel_withNormlization()
Facenet1.load_weights('MyEn3facenet.h5')
Facenet2=Transfer_FacenetModel_withNormlization()
Facenet2.load_weights('MyEn4facenet.h5')

# Transformer Learning

In [11]:
#Ensamble Learning Training

learning_rate = 0.001

weight_decay = 0.0001

batch_size = 32

num_epochs = 300

image_size = 72  # We'll resize input images to this size

patch_size = 6  # Size of the patches to be extract from the input images

num_patches = (image_size // patch_size) ** 2

projection_dim = 64

num_heads = 8

transformer_units = [

    projection_dim * 2,

    projection_dim,

]  # Size of the transformer layers

transformer_layers = 10

mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier

In [12]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [13]:
data_augmentation = keras.Sequential(

    [

        layers.experimental.preprocessing.Rescaling(1./255),

        layers.experimental.preprocessing.Resizing(image_size, image_size),

        layers.experimental.preprocessing.RandomFlip("horizontal")

        #,layers.RandomRotation(0.3)

    ],

    name="data_augmentation",

)

# Compute the mean and the variance of the training data for normalization.

#data_augmentation.layers[0].adapt(x_train)

In [14]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [15]:
def create_vit_classifier():

    inputs = layers.Input(shape=(224,224,3))

    # Augment data.

    augmented = data_augmentation(inputs)

    # Create patches.

    patches = Patches(patch_size)(augmented)

    # Encode patches.

    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)



    # Create multiple layers of the Transformer block.

    for _ in range(transformer_layers):

        # Layer normalization 1.

        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)

        # Create a multi-head attention layer.

        attention_output = layers.MultiHeadAttention(

            num_heads=num_heads, key_dim=projection_dim, dropout=0.3

        )(x1, x1)

        # Skip connection 1.

        x2 = layers.Add()([attention_output, encoded_patches])

        # Layer normalization 2.

        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)

        # MLP.

        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.3)

        # Skip connection 2.

        encoded_patches = layers.Add()([x3, x2])



    # Create a [batch_size, projection_dim] tensor.

    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)

    representation = layers.Flatten()(representation)

    representation = layers.Dropout(0.6)(representation)

    # Add MLP.

    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.6)

    # Classify outputs.

    logits = layers.Dense(5750)(features)

    # Create the Keras model.

    model = keras.Model(inputs=inputs, outputs=logits)

    return model

In [16]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [17]:
transformerClassifier1 = create_vit_classifier()
transformerClassifier2 = create_vit_classifier()

In [18]:
transformerClassifier1.load_weights('FirstTransformer3Ensamble1.h5')
transformerClassifier2.load_weights('FirstTransformer3Ensamble2.h5')

# Predictions

In [19]:
pred_prob1=Facenet1.predict(x_test)

In [20]:
pred_prob2=Facenet2.predict(x_test)

In [21]:
layer12 = tf.keras.layers.Softmax()
pred_prob3 = layer12(transformerClassifier1.predict(x_test)).numpy()

In [22]:
pred_prob4 = layer12(transformerClassifier2.predict(x_test)).numpy()

# Model Evaluation

In [23]:
from sklearn.metrics import accuracy_score
def evaluate_model(predictions):
    preds_classes = np.argmax(predictions, axis=-1)
    accuracy = accuracy_score(y_test, preds_classes)
    return accuracy*100

In [24]:
print("Accuracy for Facenet 1:", evaluate_model(pred_prob1),"%")

Accuracy for Facenet 1: 80.30418250950571 %


In [25]:
print("Accuracy for Facenet 2:", evaluate_model(pred_prob2),"%")

Accuracy for Facenet 2: 79.08745247148289 %


In [26]:
print("Accuracy for Tranformer 1:", evaluate_model(pred_prob3),"%")

Accuracy for Tranformer 1: 69.04942965779468 %


In [27]:
print("Accuracy for Tranformer 2:", evaluate_model(pred_prob4),"%")

Accuracy for Tranformer 2: 68.59315589353612 %


# Ensemble Part

## Combining Facenet models

In [28]:
print("Accuracy for Facenet ensemble:", evaluate_model(pred_prob1+pred_prob2),"%")

Accuracy for Facenet ensemble: 87.9847908745247 %


## Combining Transformer models

In [29]:
print("Accuracy for Tranformer ensemble:", evaluate_model(pred_prob3+pred_prob4),"%")

Accuracy for Tranformer ensemble: 79.46768060836501 %


## Combining all models together

In [30]:
print("Accuracy for complete ensemble:", evaluate_model(pred_prob1+pred_prob2+pred_prob3+pred_prob4),"%")

Accuracy for complete ensemble: 92.01520912547528 %


# Weighted Ensemble

In [31]:
def evaluate_weighted_ensemble(members, weights):
    # make prediction
    pred_probs = np.array(members)
    # weighted sum across ensemble members
    summed = np.tensordot(pred_probs, weights, axes=((0),(0)))
    # evaluate model
    score = evaluate_model(summed)
    return score

In [33]:
# fucntion to normalize a vector to have a unit norm
def normalize(weights):
    # calculate l1 vector norm
    result = np.linalg.norm(weights, 1)
    # check for a vector of all zeros
    if result == 0.0:
        return weights
    # else return normalized vector (unit norm)
    return weights / result

In [34]:
# function for grid search weights
from itertools import product
def grid_search(members):
    # weights to consider
    w = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    best_score, best_weights = 0.0, None
    # iterating all possible combinations using cartesian product
    for weights in product(w, repeat=len(members)):
        # skipping if all weights are equal (since already checked that case)
        if len(set(weights)) == 1:
            continue
        # hacking normalize weight vector
        weights = normalize(weights)
        # evaluate weights
        score = evaluate_weighted_ensemble(members, weights)
        if score > best_score:
            best_score, best_weights = score, weights
            print("Current best weights: ", best_weights, "with accuracy: ", best_score, "%")
    return list(best_weights)

#### grid search weights
members = [pred_prob1, pred_prob2, pred_prob3, pred_prob4]
weights = grid_search(members)
score = evaluate_weighted_ensemble(members, weights)
print('Grid Search Best Weights: {}, Accuracy: {}%'.format(weights, score))

In [42]:
def stacked_dataset(members):
    stackX = None
    for prediction in members:
        # stack predictions into [rows, members, probabilities]
        if stackX is None:
            stackX = prediction
        else:
            stackX = np.dstack((stackX, prediction))
    # flatten predictions to [rows, members x probabilities]
    stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
    return stackX

In [45]:
# fit a model based on the outputs from the ensemble 
from sklearn.linear_model import LogisticRegression
def fit_stacked_model(members, inputy):
 # create dataset using ensemble
    stackedX = stacked_dataset(members)
 # fit standalone model
    model = LogisticRegression()
    model.fit(stackedX, inputy)
    return model

In [46]:
# fit stacked model using the ensemble
model = fit_stacked_model(members, y_test)

In [47]:
# make a prediction with the stacked model
def stacked_prediction(members, model):
    # create dataset using ensemble
    stackedX = stacked_dataset(members)
     # make a prediction
    yhat = model.predict(stackedX)
    return yhat

In [ ]:
# evaluate model on test set
yhat = stacked_prediction(members, model)
acc = accuracy_score(y_test, yhat)

In [50]:
print('Stacked Test Accuracy: {}%'.format(acc*100))

Stacked Test Accuracy: 94.22053231939164%
